In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [16]:
# Load dataset
df = pd.read_csv('heart.csv')

# Preprocessing: Pisahkan fitur dan target
X = df.drop('target', axis=1).values
y = df['target'].values

In [17]:
# Normalisasi data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [18]:
# Convert ke tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [19]:
# Definisikan model MLP
def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    # Input layer to first hidden layer
    layers.append(nn.Linear(input_size, hidden_neurons))
    
    # Menambah hidden layers
    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))
    
    # Pilih fungsi aktivasi
    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()
    
    # Output layer
    layers.append(nn.Linear(hidden_neurons, 2))  # Output 2 kelas (0 atau 1)
    
    # Gabungkan semua layers
    model = nn.Sequential(*layers)
    return model

In [20]:
# Fungsi untuk melatih dan mengevaluasi model
def train_and_evaluate(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate):
    # DataLoader untuk batch training
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Optimizer (Adam) dan loss function (CrossEntropyLoss untuk klasifikasi)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    model.train()  # Set model ke mode training
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Reset gradient
            output = model(X_batch)  # Prediksi dengan model
            loss = criterion(output, y_batch)  # Hitung loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update parameter model

    # Evaluasi model setelah training
    model.eval()  # Set model ke mode evaluasi (tidak ada gradient update)
    with torch.no_grad():  # Matikan gradient untuk efisiensi
        # Training accuracy
        train_output = model(X_train_tensor)
        train_pred = torch.argmax(train_output, dim=1)  # Prediksi kelas dengan argmax
        train_accuracy = accuracy_score(y_train_tensor.numpy(), train_pred.numpy())  # Akurasi training
        train_loss = criterion(train_output, y_train_tensor).item()  # Hitung loss di data training

        # Test accuracy
        test_output = model(X_test_tensor)
        test_pred = torch.argmax(test_output, dim=1)
        test_accuracy = accuracy_score(y_test_tensor.numpy(), test_pred.numpy())  # Akurasi testing
        test_loss = criterion(test_output, y_test_tensor).item()  # Hitung loss di data testing

    return train_accuracy, train_loss, test_accuracy, test_loss  # Return hasil


In [21]:
# Daftar hyperparameter untuk eksperimen
hidden_layers_options = [1, 2, 3]  # Jumlah hidden layers
hidden_neurons_options = [8, 16, 32]  # Jumlah neuron di setiap hidden layer
activation_functions = ['linear', 'Sigmoid', 'ReLU', 'Softmax', 'Tanh']  # Fungsi aktivasi
epochs_options = [10, 25, 50, 100]  # Jumlah epochs
learning_rates = [0.1, 0.01, 0.001, 0.0001]  # Learning rates
batch_sizes = [64, 128, 256, 512]  # Ukuran batch

# Eksperimen dengan kombinasi semua hyperparameters
results = []  # Menyimpan hasil eksperimen
best_result = None
worst_result = None

for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_function in activation_functions:
            for epochs in epochs_options:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        # Print konfigurasi yang sedang dijalankan
                        print(f"Training with {hidden_layers} hidden layers, {hidden_neurons} neurons, {activation_function} activation, "
                              f"{epochs} epochs, {learning_rate} learning rate, {batch_size} batch size")

                        # Buat model untuk setiap kombinasi hyperparameter
                        model = create_mlp(X_train_tensor.shape[1], hidden_layers, hidden_neurons, activation_function)
                        
                        # Latih dan evaluasi model
                        train_acc, train_loss, test_acc, test_loss = train_and_evaluate(
                            model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate
                        )

                        # Simpan hasil eksperimen
                        results.append({
                            'Hidden Layers': hidden_layers,
                            'Hidden Neurons': hidden_neurons,
                            'Activation Function': activation_function,
                            'Epochs': epochs,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Train Accuracy': train_acc,
                            'Train Loss': train_loss,
                            'Test Accuracy': test_acc,
                            'Test Loss': test_loss
                        })

                        # Print hasil akhir akurasi dan detail lainnya
                        print(f"Train Accuracy: {train_acc * 100:.2f}%")
                        print(f"Train Loss: {train_loss:.4f}")
                        print(f"Test Accuracy: {test_acc * 100:.2f}%")
                        print(f"Test Loss: {test_loss:.4f}\n")

                        # Update best and worst results
                        if best_result is None or test_acc > best_result['Test Accuracy']:
                            best_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }
                        if worst_result is None or test_acc < worst_result['Test Accuracy']:
                            worst_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }

Training with 1 hidden layers, 8 neurons, linear activation, 10 epochs, 0.1 learning rate, 64 batch size
Train Accuracy: 85.91%
Train Loss: 0.3390
Test Accuracy: 80.84%
Test Loss: 0.3965

Training with 1 hidden layers, 8 neurons, linear activation, 10 epochs, 0.1 learning rate, 128 batch size
Train Accuracy: 86.61%
Train Loss: 0.3355
Test Accuracy: 80.52%
Test Loss: 0.4210

Training with 1 hidden layers, 8 neurons, linear activation, 10 epochs, 0.1 learning rate, 256 batch size
Train Accuracy: 85.91%
Train Loss: 0.3356
Test Accuracy: 78.90%
Test Loss: 0.4216

Training with 1 hidden layers, 8 neurons, linear activation, 10 epochs, 0.1 learning rate, 512 batch size
Train Accuracy: 86.19%
Train Loss: 0.3421
Test Accuracy: 80.84%
Test Loss: 0.4367

Training with 1 hidden layers, 8 neurons, linear activation, 10 epochs, 0.01 learning rate, 64 batch size
Train Accuracy: 86.47%
Train Loss: 0.3341
Test Accuracy: 80.19%
Test Loss: 0.4200

Training with 1 hidden layers, 8 neurons, linear activat

In [22]:
# Menampilkan best dan worst hyperparameter berdasarkan akurasi
print("\nBest Hyperparameter Configuration:")
print(f"Hidden Layers: {best_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {best_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {best_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {best_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {best_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {best_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {best_result['Test Accuracy'] * 100:.2f}%")

print("\nWorst Hyperparameter Configuration:")
print(f"Hidden Layers: {worst_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {worst_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {worst_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {worst_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {worst_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {worst_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {worst_result['Test Accuracy'] * 100:.2f}%")


Best Hyperparameter Configuration:
Hidden Layers: 3
Hidden Neurons: 32
Activation Function: Softmax
Epochs: 25
Learning Rate: 0.1
Batch Size: 512
Test Accuracy: 84.09%

Worst Hyperparameter Configuration:
Hidden Layers: 2
Hidden Neurons: 32
Activation Function: Sigmoid
Epochs: 10
Learning Rate: 0.1
Batch Size: 256
Test Accuracy: 19.81%
